<a href="https://colab.research.google.com/github/vithushanms/makemore-language-model/blob/main/%5BLanguage_Models%5D_makemore_MLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [48]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt # for making figures
%matplotlib inline

In [8]:
#if cloud
from google.colab import drive
drive.mount('/content/drive')
dataFilePath = '/content/drive/My Drive/Publications/Neural Networks Research/makemore: next char prediction language model/names.txt'
words = open(dataFilePath, 'r').read().splitlines() 

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
len(words)

32033

In [10]:
# encode chars in the words to integer
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [54]:
[0] * 3

[0, 0, 0]

In [55]:
# build the dataset

'''
build the dataset while having the block_size (the number of chars we are going to input at a time) as 3
'''
block_size = 3 
X, Y = [], []
for w in words[:5]:
  print(w)
  context = [0] * block_size
  for ch in w + '.':
    ix = stoi[ch]
    X.append(context)
    Y.append(ix)
    print(''.join(itos[i] for i in context), '--->', itos[ix])
    context = context[1:] + [ix]
  
X = torch.tensor(X)
Y = torch.tensor(Y)

emma
... ---> e
..e ---> m
.em ---> m
emm ---> a
mma ---> .
olivia
... ---> o
..o ---> l
.ol ---> i
oli ---> v
liv ---> i
ivi ---> a
via ---> .
ava
... ---> a
..a ---> v
.av ---> a
ava ---> .
isabella
... ---> i
..i ---> s
.is ---> a
isa ---> b
sab ---> e
abe ---> l
bel ---> l
ell ---> a
lla ---> .
sophia
... ---> s
..s ---> o
.so ---> p
sop ---> h
oph ---> i
phi ---> a
hia ---> .


In [75]:
X[:5]

tensor([[ 0,  0,  0],
        [ 0,  0,  5],
        [ 0,  5, 13],
        [ 5, 13, 13],
        [13, 13,  1]])

In [77]:
Y[:5]

tensor([ 5, 13, 13,  1,  0])

In [78]:
X.shape

torch.Size([32, 3])

Basically matrix multiplication with the correct dimension with ont hot encoded matrix and indexing the integer value itself will give the same result 

In [70]:
C = torch.rand(27,3)

In [71]:
#indexing|
C[2]

tensor([0.7617, 0.7973, 0.4204])

In [72]:
#one hot enoding 
five_en = torch.nn.functional.one_hot(torch.tensor(2), num_classes=27).float()
five_en

tensor([0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [73]:
# multiplication [1 x 27] * [27, 2]
five_en @ C

tensor([0.7617, 0.7973, 0.4204])

So we can dierectly use indexing here

In [69]:
# we can index the whole data set at one go
C[X]

torch.Size([32, 3, 3])

In [79]:
C[X].shape

torch.Size([32, 3, 3])

In [34]:
emb = C[X]

In [39]:
torch.cat([emb[: , 0, :], emb[:, 1, :], emb[:, 2, :]], dim=1).shape

torch.Size([32, 6])

In [40]:
W1 = torch.randn((6,100))
b = torch.randn(100)

In [42]:
out = emb.view(32,6) @ W1 + b

In [80]:
out.shape

torch.Size([32, 100])